In [11]:
import builtwith
import whois
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

### Página web

In [4]:
URL = 'https://tarifaluzhora.es/'

### Evaluación inicial

In [8]:
# Check robots.txt
url_robots = URL + '/robots.txt'
page = requests.get(url_robots)
print(page.text)

#
# robots.txt
#
# This file is to prevent the crawling and indexing of certain parts
# of your site by web crawlers and spiders run by sites like Yahoo!
# and Google. By telling these "robots" where not to go on your site,
# you save bandwidth and server resources.
#
# This file will be ignored unless it is at the root of your host:
# Used:    http://example.com/robots.txt
# Ignored: http://example.com/site/robots.txt
#
# For more information about the robots.txt standard, see:
# http://www.robotstxt.org/robotstxt.html

User-agent: *
# CSS, JS, Images
Allow: /core/*.css$
Allow: /core/*.css?
Allow: /core/*.js$
Allow: /core/*.js?
Allow: /core/*.gif
Allow: /core/*.jpg
Allow: /core/*.jpeg
Allow: /core/*.png
Allow: /core/*.svg
Allow: /profiles/*.css$
Allow: /profiles/*.css?
Allow: /profiles/*.js$
Allow: /profiles/*.js?
Allow: /profiles/*.gif
Allow: /profiles/*.jpg
Allow: /profiles/*.jpeg
Allow: /profiles/*.png
Allow: /profiles/*.svg
# Directories
Disallow: /core/
Disallow: /profiles/
# Fil

Observamos en el archivo robots.txt que para cualquier rastreador se habilitan todos los elementos gráficos, código y estilos y lo que se deshabilita son la parte para registro de usuarios y configuraciones de la web.

In [9]:
# Tecnologia del Sitio
print(builtwith.builtwith(URL))

{'web-servers': ['Nginx'], 'tag-managers': ['Google Tag Manager'], 'web-frameworks': ['Twitter Bootstrap']}


In [10]:
# Propietario del Sitio
print(whois.whois(URL))

{
  "domain_name": null,
  "registrar": null,
  "whois_server": null,
  "referral_url": null,
  "updated_date": null,
  "creation_date": null,
  "expiration_date": null,
  "name_servers": null,
  "status": null,
  "emails": null,
  "dnssec": null,
  "name": null,
  "org": null,
  "address": null,
  "city": null,
  "state": null,
  "zipcode": null,
  "country": null
}


### Scraping por fecha

In [26]:
def scrap_data(tipo_tarifa,d,m,y):
    
    url = 'https://tarifaluzhora.es/?tarifa=' + tipo_tarifa + '&fecha=' + d + '%2F' + m + '%2F' + y

    # Obtenemos página
    raw_html = requests.get(url).text

    # La parseamos
    data = BeautifulSoup(raw_html, 'html.parser')
    
    #for i in range (50):
      #print(i+1,":", data.select('span')[i])
    
    # Generamos dataframe mientras buscamos datos y los limpiamos
    df = pd.DataFrame()

    df['Fecha'] = [data.select("span")[6].string]
    df['Franja más cara'] = [data.select("span")[9].string]
    df['Franja más barata'] = [data.select("span")[7].string]

    tags_hora=data.find_all(itemprop="description")    
    tags_precio=data.find_all(itemprop="price")

    a=zip(tags_hora, tags_precio)

    for i,j in a:
        df[i.string] = [j.string.replace(' €/kWh','')]
    
    return df

# TODAY
#today = datetime.datetime.now()
#day = today.strftime('%d') 
#month= today.strftime('%m')
#year = str(today.year)

# Tarifas: normal, discriminacion (discriminacion horaria), coche_electrico (vehiculos electricos)
tarifa = 'normal'

dataset = scrap_data(tarifa, '05', '01', '2021')

In [27]:
dataset

,Fecha,Franja más cara,Franja más barata,00h - 01h:,01h - 02h:,02h - 03h:,03h - 04h:,04h - 05h:,05h - 06h:,06h - 07h:,...,14h - 15h:,15h - 16h:,16h - 17h:,17h - 18h:,18h - 19h:,19h - 20h:,20h - 21h:,21h - 22h:,22h - 23h:,23h - 24h:
0,05 de 01 de 2021,18h - 19h,04h - 05h,0.13822,0.12873,0.12408,0.12325,0.12275,0.12322,0.12962,...,0.1345,0.13319,0.13516,0.1379,0.14805,0.14761,0.14328,0.14324,0.14238,0.13956
